In [1]:
import pandas as pd
from queue import Queue

In [17]:
def update_ftr(fthg, ftag):
    # corrigir o resultado (W/L/D)
    if fthg < ftag:
        return 'A'
    
    elif fthg == ftag:
        return 'D'
    else:
        return 'H'

In [18]:
def update_goals_points(s, goals, points ):
    
    # jogo de pre-epoca
    # campeonato apenas começa na week 2
    HomeTeam = s['HomeTeam']
    AwayTeam = s['AwayTeam']
    
    if s['MW'] == 1.0:
        # reset dos valores
        
        #goals = dict((team, 0) for team in diff_teams)
        #points = dict((team, 0) for team in diff_teams)
        
        goals[HomeTeam] = 0
        goals[AwayTeam] = 0
        points[HomeTeam] = 0
        points[AwayTeam] = 0
        

    else:
        diff = s['FTHG'] - s['FTAG']
        
        goals[HomeTeam]+= diff
        goals[AwayTeam]+= -diff
        
        if diff < 0: # equipa visitante ganhou
            points[AwayTeam]+= 3
            
            
    
        elif diff == 0: # empate
            
            points[HomeTeam]+= 1
            points[AwayTeam]+= 1
            
            
        
        else: # equipa da casa ganhou
            
            points[HomeTeam]+= 3
            
    return pd.Series([goals[HomeTeam], goals[AwayTeam], points[HomeTeam], points[AwayTeam]],index=['HTGD','ATGD','HTP', 'ATP'])

In [2]:


data = pd.read_csv('final_dataset.csv')

df = pd.DataFrame(data, columns=['HomeTeam','AwayTeam','HTGS', 'ATGS', 'HTGC', 'ATGC', 
                                 'FTHG','FTAG', 'FTR','HomeTeamLP', 'AwayTeamLP',
                                  'HTFormPts','ATFormPts', 'HTGD', 'ATGD', 'MW', 'HTP', 'ATP'])

df['FTR'] = df.apply(lambda x : update_ftr(x.FTHG, x.FTAG), axis=1)

diff_teams = set(df['HomeTeam'].values) # todas as equipas

goals = dict((team, 0) for team in diff_teams) # differeça de golos de cada equipa
points = dict((team, 0) for team in diff_teams) # pontos de cada equipa

print(goals)

df[['HTGD','ATGD','HTP', 'ATP']] = df.apply(lambda x : update_goals_points(x, goals, points), axis=1)


# escrever em csv
df.to_csv('premier_train_dataset.csv')
        

{'Chelsea': 0, 'Man City': 0, 'Watford': 0, 'Arsenal': 0, 'Burnley': 0, 'Blackpool': 0, 'Man United': 0, 'Leicester': 0, 'Everton': 0, 'Stoke': 0, 'Bournemouth': 0, 'Derby': 0, 'Norwich': 0, 'Birmingham': 0, 'Blackburn': 0, 'Charlton': 0, 'Coventry': 0, 'Fulham': 0, 'Crystal Palace': 0, 'Newcastle': 0, 'Bolton': 0, 'Sunderland': 0, 'Ipswich': 0, 'West Brom': 0, 'Portsmouth': 0, 'Wolves': 0, 'Sheffield United': 0, 'Hull': 0, 'QPR': 0, 'Southampton': 0, 'Tottenham': 0, 'Wigan': 0, 'Aston Villa': 0, 'Middlesboro': 0, 'Reading': 0, 'Leeds': 0, 'Middlesbrough': 0, 'West Ham': 0, 'Cardiff': 0, 'Liverpool': 0, 'Swansea': 0, 'Bradford': 0}


In [40]:
def update_test_data(data):
    
    #select features
    df = pd.DataFrame(data, columns=['HomeTeam','AwayTeam','HTGS', 'ATGS', 'HTGC', 'ATGC', 
                                 'FTHG','FTAG', 'FTR','HTFormPts','ATFormPts',
                                  'HTGD', 'ATGD', 'HTP', 'ATP'])
    
    diff_teams = set(df['HomeTeam'].values) # todas as equipas
    
    gs = dict((team, 0) for team in diff_teams) # golos marcados
    gc = gs.copy() # golos sofridos
    goals = gs.copy() # differeça de golos de cada equipa
    points = gs.copy() # pontos de cada equipa


    
    form = dict((team, Queue(maxsize = 5)) for team in diff_teams) # forma dos ultimos 5 jogos
    
    df[['HTGS', 'ATGS', 'HTGC','ATGC', 'HTFormPts', 
        'ATFormPts','HTGD','ATGD', 'HTP', 'ATP']] = df.apply(lambda x : update_test_params(x, gs, gc ,form ,points  ), axis=1)
    
    return df
    
    

In [43]:
def update_test_params(s, gs, gc, form, points ):
    
    # jogo de pre-epoca
    # campeonato apenas começa na week 2
    HomeTeam = s['HomeTeam']
    AwayTeam = s['AwayTeam']
    

    diff = s['FTHG'] - s['FTAG']
    
    
    gs[HomeTeam]+=s['FTHG']
    gc[HomeTeam]+=s['FTAG']
    gs[AwayTeam]+=s['FTAG']
    gc[AwayTeam]+=s['FTAG']
    
    
    if form[HomeTeam].full():
        form[HomeTeam].get()
        
    if form[AwayTeam].full():
        form[AwayTeam].get()
        
        
    if diff < 0: # equipa visitante ganhou
        form[HomeTeam].put(3)
        form[AwayTeam].put(0)
        points[HomeTeam]+=3

    elif diff == 0: # empate

        form[HomeTeam].put(1)
        form[AwayTeam].put(1)
        points[HomeTeam]+=1
        points[AwayTeam]+=1

    else: # equipa da casa ganhou

        form[AwayTeam].put(3)
        form[HomeTeam].put(0)
        points[AwayTeam]+=3
        
    htgd= gs[HomeTeam] - gc[HomeTeam]
    atgd = gs[AwayTeam] - gc[AwayTeam]
    
    return pd.Series([gs[HomeTeam], gs[AwayTeam], gc[HomeTeam], gc[AwayTeam],
                      sum(list(form[HomeTeam].queue)), sum(list(form[AwayTeam].queue)),htgd ,atgd ,points[HomeTeam], points[AwayTeam] ],
                     index=['HTGS', 'ATGS', 'HTGC','ATGC', 'HTFormPts', 'ATFormPts','HTGD','ATGD', 'HTP', 'ATP'])

In [46]:




# juntar dataset para teste
data_train = pd.concat([update_test_data(pd.read_csv('2017-2018.csv'))
                        , update_test_data(pd.read_csv('2018-2019.csv'))
                        , update_test_data(pd.read_csv('2019-2020.csv'))
                        , update_test_data(pd.read_csv('2020-2021.csv'))])


data_train.to_csv('premier_test_dataset.csv')

